# Prepares

In [ ]:
import os 

In [ ]:
input_path = 'spinn_hmi'
input_original = os.path.join(input_path, 'original')
os.makedirs(input_original, exist_ok=True)

In [ ]:
bin = 2
b_norm = 2500
spatial_norm = 160
nz = 160

## SPINN

In [ ]:
import pickle

In [ ]:
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "0"

In [ ]:
import jax 
import jax.numpy as jnp
from jax import jvp
import optax
from flax import linen as nn 

from typing import Sequence
from functools import partial

import numpy as np

import time
from tqdm import trange

In [ ]:
input_original = os.path.join(input_path, 'original')

b_bottom_path = os.path.join(input_path, "b_bottom.pickle")
bp_top_path = os.path.join(input_path, "bp_top.pickle")
bp_lateral_1_path = os.path.join(input_path, "bp_lateral_1.pickle")
bp_lateral_2_path = os.path.join(input_path, "bp_lateral_2.pickle")
bp_lateral_3_path = os.path.join(input_path, "bp_lateral_3.pickle")
bp_lateral_4_path = os.path.join(input_path, "bp_lateral_4.pickle")

In [ ]:
with open(b_bottom_path,"rb") as f:
    b_bottom = pickle.load(f)

with open(bp_top_path,"rb") as f:
    bp_top = pickle.load(f)

with open(bp_lateral_1_path,"rb") as f:
    bp_lateral_1 = pickle.load(f)

with open(bp_lateral_2_path,"rb") as f:
    bp_lateral_2 = pickle.load(f)

with open(bp_lateral_3_path,"rb") as f:
    bp_lateral_3 = pickle.load(f)

with open(bp_lateral_4_path,"rb") as f:
    bp_lateral_4 = pickle.load(f)

In [ ]:
seed = 111
key = jax.random.PRNGKey(seed)
key, subkey = jax.random.split(key, 2)

In [ ]:
features = 256
n_layers = 8 
feat_sizes = tuple([features for _ in range(n_layers)]) 
r = 128 
out_dim = 3 

lr = 5e-4

epochs = 2000
log_iter = 100

In [ ]:
from zpinn.spinn_cleanup_new import SPINN3d, generate_train_data, apply_model_spinn, update_model

In [ ]:
b_bottom.shape

(344, 224, 3)

In [ ]:
nx, ny, _ = b_bottom.shape

In [ ]:
model = SPINN3d(feat_sizes, r, out_dim, pos_enc=0, mlp='modified_mlp')
params = model.init(
            subkey,
            jnp.ones((nx, 1)),
            jnp.ones((ny, 1)),
            jnp.ones((nz, 1))
        )
apply_fn = jax.jit(model.apply)
optim = optax.adam(learning_rate=lr)
state = optim.init(params)

In [ ]:
key, subkey = jax.random.split(key, 2)
train_data = generate_train_data(nx, ny, nz, key)

In [ ]:
boundary_data = (b_bottom, bp_top, bp_lateral_1, bp_lateral_2, bp_lateral_3, bp_lateral_4)

In [ ]:
train_boundary_data = [train_data, boundary_data]

In [ ]:
loss, gradient = apply_model_spinn(apply_fn, params, train_boundary_data)
params, state = update_model(optim, gradient, params, state)

In [ ]:
result_path = 'spinn_hmi/output'
os.makedirs(result_path, exist_ok=True)

In [ ]:
start = time.time()
for e in trange(1, epochs + 1):
    
    # if e % 300 == 0:
    #     # sample new input data
    #     key, subkey = jax.random.split(key, 2)
    #     train_data = generate_train_data(nx, ny, nz, subkey)

    loss, gradient = apply_model_spinn(apply_fn, params, train_boundary_data)
    params, state = update_model(optim, gradient, params, state)
    
    if e % log_iter == 0 or e == 1:
        print(f'Epoch: {e}/{epochs} --> total loss: {loss:.8f}')
        params_path = os.path.join(result_path, f"params_{e}.pickle")
        with open(params_path, "wb") as f:
            pickle.dump(params, f)

runtime = time.time() - start
print(f'Runtime --> total: {runtime:.2f}sec ({(runtime/(epochs-1)*1000):.2f}ms/iter.)')

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 1/2000 [00:00<03:43,  8.96it/s]

Epoch: 1/2000 --> total loss: 0.04490991


  5%|▌         | 103/2000 [00:09<02:44, 11.52it/s]

Epoch: 100/2000 --> total loss: 0.03350798


...

Epoch: 1800/2000 --> total loss: 0.01283587


 95%|█████████▌| 1903/2000 [02:48<00:08, 11.11it/s]

Epoch: 1900/2000 --> total loss: 0.01246905


100%|██████████| 2000/2000 [02:57<00:00, 11.27it/s]

Epoch: 2000/2000 --> total loss: 0.01246227
Runtime --> total: 177.40sec (88.75ms/iter.)
